In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
# change the dataframes column names and datatypes
def parse_data(df):
    return(
        df.rename(columns={
            "<Ticker>":"ticker",
            '<DTYYYYMMDD>':'date', 
            '<Open>':'open', 
            '<High>':'high', 
            '<Low>':'low', 
            '<Close>':'close',
            '<Volume>':'volume'
        }).assign(
            date = lambda x: pd.to_datetime(x['date'], format="%Y%m%d").dt.date
        )
    )

In [24]:
# Read data
hnx = parse_data(pd.read_csv(os.path.join('data', "CafeF.HNX.Upto29.05.2025.csv")))
hsx = parse_data(pd.read_csv(os.path.join('data', "CafeF.HSX.Upto29.05.2025.csv")))
upcom = parse_data(pd.read_csv(os.path.join('data', "CafeF.UPCOM.Upto29.05.2025.csv")))

In [25]:
upcom

,ticker,date,open,high,low,close,volume
0,AAH,2025-05-29,4.2000,4.2000,4.0000,4.2000,1369500
1,AAS,2025-05-29,9.5000,9.5000,8.8000,9.0000,226000
2,ABB,2025-05-29,8.1000,8.3000,8.0000,8.1000,3498100
3,ABC,2025-05-29,11.0000,11.5000,11.0000,11.3000,92700
4,ABI,2025-05-29,27.9000,28.2000,27.8000,28.2000,143400
...,...,...,...,...,...,...,...
766623,BTC,2002-01-30,13.5251,13.5251,13.5251,13.5273,39400
766624,BTC,2002-01-28,13.5251,13.5251,13.5251,13.5273,800
766625,BTC,2002-01-25,13.2909,13.2909,13.2909,13.2931,100
766626,BTC,2002-01-23,13.0567,13.0567,13.0567,13.0588,19000
